## benchmarks for flowsom algorithm, searching for improvements

In [2]:
import timeit
from collections.abc import Callable
from memory_profiler import memory_usage

# imports
import numpy as np
from flowio import FlowData
from sklearn.metrics import v_measure_score

import flowsom as fs

### code from docker to filter out NaN values

In [3]:


def read_labelled_fcs(path,label_column=-1):
    # read in FCS file
    fcs_data = FlowData(path)
    # convert to numpy array
    npy_data = np.reshape(fcs_data.events, (-1, fcs_data.channel_count))
    # get label column
    # TODO: support more files
    y = npy_data[:, label_column]
    # filter out unlabelled data
    mask = ~np.isnan(y)
    X = npy_data[mask, :-1]
    y = npy_data[mask, label_column]
    # if no 0 in y, subtract 1 from all labels
    # this is to make sure that the labels start at 0, as sklearn clustering algorithms usually output
    if 0 not in y:
        y = y - 1
    # cast y to int
    y = y.astype(np.int32)
    return X, y


def score_fcs_file(path, flowsom_func,dimensions,cols_to_use,seed) -> float:
    # read in fcs file
    X, y = read_labelled_fcs(path)

    # finding the best number of clusters is not part of this test
    # here we use labelled data to find the number of unique labels
    n_clusters = np.unique(y).shape[0]
    
    # cluster data and predict labels
    fsom = flowsom_func(X, n_clusters = max(n_clusters, dimensions, len(cols_to_use)),xdim=10, ydim=10,cols_to_use=cols_to_use,seed=seed)
    y_pred = fsom.metacluster_labels

    # because the v_measure_score is independent of the absolute values of the labels
    # we don't need to make sure the predicted label values have the same value as the true labels
    # the v_measure_score will be the same regardless, as it only depends on homogeneity and completeness
    # alternatively, a lookup table from the cluster centers can be used to have a consistent label value mapping
    # https://stackoverflow.com/questions/44888415/how-to-set-k-means-clustering-labels-from-highest-to-lowest-with-python
    v_measure = v_measure_score(y, y_pred)
    print(f"V-measure score: {v_measure}")
    return v_measure

In [13]:
# simple time benchmark

ff = fs.io.read_FCS("../data/accuracy_benches/Samusik_all.fcs")
ff.uns['meta']['channels']

,$PnN,$PnS,$PnB,$PnE,$PnR
n,,,,,
1,Time,,32,"0,0",1024
2,Cell_length,,32,"0,0",1024
3,BC1,,32,"0,0",1024
4,BC2,,32,"0,0",1024
5,BC3,,32,"0,0",1024
6,BC4,,32,"0,0",1024
7,BC5,,32,"0,0",1024
8,BC6,,32,"0,0",1024
9,Ter119,,32,"0,0",1024


In [ ]:
ff.var

### put together array with markers used

### testing flowsom

In [5]:
cols = list(range(13))
score_fcs_file("../data/accuracy_benches/Levine_13dim.fcs",fs.FlowSOM,13,cols,None)

2024-05-27 11:47:11.369 | DEBUG    | flowsom.main:__init__:84 - Reading input.
2024-05-27 11:47:11.385 | DEBUG    | flowsom.main:__init__:86 - Fitting model: clustering and metaclustering.


SOM training took: 2.120572 seconds.


2024-05-27 11:47:14.349 | DEBUG    | flowsom.main:__init__:88 - Updating derived values.


V-measure score: 0.8200886214874876


0.8200886214874876

### Make benchmarking function for time as well as peak memory

In [6]:
def bench_file(path:str, flowsom_implementation, dimensions:int, label_col=-1,cols_to_use:np.ndarray=None, seed:int=None):
    """
    Benchmark a file with the given implementation of flowsom, this includes time and v-measure score
    @param path: path to the fcs file
    @param flowsom_implementation: implementation of flowsom to use
    @param dimensions: number of dimensions to use
    @param cols_to_use: columns to use
    @param seed: random seed to use
    """
    # read in fcs file
    X, y = read_labelled_fcs(path,label_col)
    
    # finding the best number of clusters is not part of this test
    # here we use labelled data to find the number of unique labels
    n_clusters = np.unique(y).shape[0]

    # cluster data and predict labels
    fsom = []
    exec_time = timeit.timeit(lambda: fsom.append(flowsom_implementation(X, n_clusters = max(n_clusters, dimensions, len(cols_to_use)), xdim=10, ydim=10, cols_to_use=cols_to_use, seed=seed)),number=1)
    y_pred = fsom[0].metacluster_labels
    
    # Measure peak memory usage
    peak_memory = max(memory_usage(proc=(lambda: flowsom_implementation(X, n_clusters = max(n_clusters, dimensions, len(cols_to_use)), xdim=10, ydim=10, cols_to_use=cols_to_use, seed=seed)), interval=0.1))
    
    
    # because the v_measure_score is independent of the absolute values of the labels
    # we don't need to make sure the predicted label values have the same value as the true labels
    # the v_measure_score will be the same regardless, as it only depends on homogeneity and completeness
    # alternatively, a lookup table from the cluster centers can be used to have a consistent label value mapping
    # https://stackoverflow.com/questions/44888415/how-to-set-k-means-clustering-labels-from-highest-to-lowest-with-python
    v_measure = v_measure_score(y, y_pred)
    print(f"V-measure score: {v_measure}")
    print(f'Execution time: {exec_time}s')
    print(f"Peak memory usage: {peak_memory:.2f} MiB")
    return (v_measure,exec_time,peak_memory)


In [10]:
cols = list(range(13))
bench_file("../data/accuracy_benches/Levine_13dim.fcs",fs.FlowSOM,13,cols_to_use=cols)

2024-05-27 12:00:47.967 | DEBUG    | flowsom.main:__init__:84 - Reading input.
2024-05-27 12:00:47.987 | DEBUG    | flowsom.main:__init__:86 - Fitting model: clustering and metaclustering.


SOM training took: 0.883817 seconds.


2024-05-27 12:00:49.181 | DEBUG    | flowsom.main:__init__:88 - Updating derived values.
2024-05-27 12:00:49.502 | DEBUG    | flowsom.main:__init__:84 - Reading input.
2024-05-27 12:00:49.523 | DEBUG    | flowsom.main:__init__:86 - Fitting model: clustering and metaclustering.


SOM training took: 0.883158 seconds.


2024-05-27 12:00:50.730 | DEBUG    | flowsom.main:__init__:88 - Updating derived values.


V-measure score: 0.8540181895789065
Execution time: 1.4985259909990418s
Peak memory usage: 1192.49 MiB


(0.8540181895789065, 1.4985259909990418, 1192.48828125)

### now test different improvements

In [8]:
# cols = list(range(2,12))
# bench_file("../data/accuracy_benches/FlowCAP_ND.fcs",fs.FlowSOM,10,-2,cols)